In [1]:
from matplotlib import pyplot as plt
import numpy as np
import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath("__file__")))
from case_studies import *

import matplotlib
matplotlib.use('agg')

In [2]:
def wolfe_search(f, df, x, p, alpha0, c1, c2):
    """
    Implements the Wolfe line-search algorithm (Algorithm 8) which returns a
    step length fulfilling the strong Wolfe conditions.

    Arguments:
        f      : objective function f(x)
        df     : gradient function df(x)
        x      : current iterate
        p      : search direction
        alpha0 : initial upper bracket / step-length guess
        c1, c2 : Wolfe condition parameters (0 < c1 < c2 < 1)

    Returns:
        alpha    : step length satisfying the strong Wolfe conditions
        brackets : list of [l, u] brackets recorded at each iteration
    """
    # Scalar wrappers: g(a) = f(x + a*p),  g'(a) = ∇f(x+a*p)ᵀp
    def g(a):
        return f(x + a * p)

    def dg(a):
        return np.inner(df(x + a * p), p)

    l = 0
    u = alpha0
    brackets = [[l, u]]   # record initial bracket (iteration 1)

    # ── Phase 1: expansion ────────────────────────────────────────────
    while True:
        if g(u) > g(0) + c1 * u * dg(0) or g(u) > g(l):
            break
        if abs(dg(u)) < c2 * abs(dg(0)):
            return u, brackets
        if dg(u) > 0:
            break
        else:
            u = u * 2
            brackets.append([l, u])

    # ── Phase 2: bisection ────────────────────────────────────────────
    while True:
        brackets.append([l, u])
        a = (l + u) / 2
        if g(a) > g(0) + c1 * a * dg(0) or g(a) > g(l):
            u = a
        else:
            if abs(dg(a)) < c2 * abs(dg(0)):
                return a, brackets
            if dg(a) < 0:
                l = a
            else:
                u = a